In [ ]:
##############################################################################
#
#            Requirements:
#              - Python 3.5+
#              - Exiv2 installation
# 
##############################################################################

def read_config_values():
    """Reads values from the configuration files and returns dictionary"""
    
    if config_file_exists():
        import image_caption_config
    else:
        raise ValueError ('Missing file image_caption_config.py, please refer to documentation') 
    
    # Values in a hidden config file override the config file, useful to prevent user config settings being pushed to github
    if hidden_config_file_exists():
        import _image_caption_config
        if image_caption_config.config.keys() != _image_caption_config.config.keys():
            raise ValueError ('keys in config files dont match hidden config file') 
        else:
            config_values = _image_caption_config.config
    else:
        config_values = image_caption_config.config
    
    return (config_values)





        
if __name__ == "__main__":
    

  
    # Read configuration inputs into a dictionary
    inputs = read_config_values()
    
    if not dir_exists(inputs['out_dir']):
        create_dir_and_parents(inputs['out_dir'])
    elif not dir_is_empty(inputs['out_dir']):
        raise ValueError ('Output directory (' +  inputs['out_dir'] +') is not empty') 
    
    for filename_and_extension in os.listdir(inputs['in_dir']):
        image_extensions_lower_case = ['.jpeg', '.jpg', '.png']
        base_filename, file_extension = os.path.splitext(filename_and_extension)        
        if file_extension.lower() in image_extensions_lower_case:
            input_filepath = os.path.join(inputs['in_dir'], base_filename + file_extension)
            output_filepath = os.path.join(inputs['out_dir'], base_filename + file_extension)
            print ('processing ' + input_filepath)
            image_title = get_image_title(input_filepath)
            if title_is_valid_for_caption(image_title, inputs['caption_prefix']):
                input_image = get_input_image(input_filepath)
                caption = image_title_ex_prefix(image_title, inputs['caption_prefix'])    
                output_image= generate_captioned_image(input_image=input_image, frame_ratio = inputs['frame_ratio'], 
                    border_ratio = inputs['border_ratio'], border_colour = inputs['border_colour'], 
                    frame_colour = inputs['frame_colour'],caption_font = inputs['caption_font'], 
                    font_ratio = inputs['font_ratio'], font_size_min = inputs['font_size_min'],                                   
                    font_colour = inputs['font_colour'], caption = caption)
                output_image.save(output_filepath)
                copy_metadata(input_filepath, output_filepath)
                
            else:
                shutil.copy2(input_filepath, output_filepath)
            
            

    print('\nProcessing complete...')